In [1]:
import polars as pl

In [2]:
from IPython import get_ipython
get_ipython().run_line_magic("run", "functions.ipynb")

In [13]:
cierre = "202503"
cierre_l = "Mar25"

# Carpetas
path = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = path + "data/raw/"
wd_data_external = path + "data/external/"
wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/DWH/csv/"


# Archivos
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}.accdb'
fl_saldos = wd_data_raw + f"Saldos_{cierre}.mdb"
fl_saldos_bmxt = wd_data_raw + f"Saldos_BMXT_{cierre}.mdb"
fl_saldos_fianzas = wd_data_raw + f"Saldos_80686_{cierre}.mdb"

fl_saldos_totales_vf = wd_data_processed_dwh + f'DWH_Saldos_Totales_{cierre}_VF.csv'
fl_agrup_saldo_total = wd_data_processed_dwh + f'BD_DWH_Agrup_SaldoTotal_{cierre}.csv'
fl_agrup_taxo_saldo_total = wd_data_processed_dwh + f'BD_DWH_AgrupTaxo_SaldoTotal_{cierre}.csv'

In [7]:
schema_saldos = {
    'DESC_INDICADOR': pl.String,
    'FECHA_CONSULTA': pl.Date,
    'HISTORICO': pl.String,
    'INDICADOR_ID': pl.Int32,
    'INTERMEDIARIO_ID': pl.String,
    'PROGRAMA_DESCRIPCION': pl.String,
    'PROGRAMA_ID': pl.Int32,
    'PROGRAMA_ORIGINAL': pl.Int32,
    'RAZON_SOCIAL': pl.String,
    'TIPO_CREDITO_ID': pl.Int16,
    'Número de registros': pl.Int16,
    'SALDO_CONTINGENTE_MN (SUMA)': pl.Float64
}

In [8]:
def import_saldos_totales():
    df_saldos = read_access(fl_saldos, "DATOS")
    df_saldos_bmxt = read_access(fl_saldos_bmxt, "DATOS")
    df_saldos_fianzas = read_access(fl_saldos_fianzas, "DATOS")

    df_saldos_totales = pl.concat([
        df_saldos, 
        df_saldos_bmxt, 
        df_saldos_fianzas
        ], rechunk=True)
    
    df_saldos_totales = (
        df_saldos_totales.select(
            pl.col("DESC_INDICADOR").alias("Producto"),
            pl.col("FECHA_CONSULTA"),
            pl.col("INDICADOR_ID").alias("Producto ID"),
            pl.col("INTERMEDIARIO_ID"),
            pl.col("PROGRAMA_DESCRIPCION").alias("Programa Descripción (ID)"),
            pl.col("PROGRAMA_ID"),
            pl.col("PROGRAMA_ORIGINAL"),
            pl.col("RAZON_SOCIAL").alias("Razón Social (Intermediario)"),
            pl.col("TIPO_CREDITO_ID"),
            pl.col("SALDO_CONTINGENTE_MN (SUMA)").alias("SUM Saldo_Contingente_Mn")
        )
    )
    df_saldos_totales = genera_tpro_clave(df_saldos_totales)
    return df_saldos_totales

df_saldos_totales = import_saldos_totales()

In [9]:
def genera_saldos_totales_vf(df):
    tipo_credito = read_access(fl_catalogos, "TIPO_CREDITO")
    tipo_credito = tipo_credito.cast({'Tipo_Credito_ID': pl.Int32})

    df_saldos_totales_vf = (
        df.join(tipo_credito.select(['Tipo_Credito_ID', 'NR_R']),
                            left_on='TIPO_CREDITO_ID', right_on='Tipo_Credito_ID',
                            how='left')
    )
    return df_saldos_totales_vf

saldos_totales_vf = genera_saldos_totales_vf(df_saldos_totales)


In [12]:
saldos_totales_vf.write_csv(fl_saldos_totales_vf, datetime_format="%Y-%m-%d", float_precision=6)

In [17]:
# Agrup saldo total
agrup_saldo_total = (
    saldos_totales_vf
    .group_by([
        "Producto", 
        "Razón Social (Intermediario)", 
        "INTERMEDIARIO_ID", 
        "PROGRAMA_ORIGINAL", 
        "PROGRAMA_ID"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)
agrup_saldo_total.write_csv(fl_agrup_saldo_total, datetime_format="%Y-%m-%d", float_precision=6)

In [20]:
# Agrup saldo total resto
(
    saldos_totales_vf
    .filter(
        (pl.col("Producto") == "GARANTIAS BANCOMEXT") |
        (pl.col("Producto") == "GARANTIAS SHF/LI FINANCIERO") |
        (pl.col("Producto") == "GARANTIAS BANSEFI") |
        (pl.col("Razón Social (Intermediario)") == "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
    ).group_by([
        "Producto", 
        "Razón Social (Intermediario)", 
        "INTERMEDIARIO_ID", 
        "PROGRAMA_ORIGINAL", 
        "PROGRAMA_ID"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,Razón Social (Intermediario),INTERMEDIARIO_ID,PROGRAMA_ORIGINAL,PROGRAMA_ID,SALDO_MN
str,str,str,i64,i64,f64
"""GARANTIAS BANCOMEXT""","""BANCOMEXT BBVA BANCOMER""","""60423958""",3999,39698,1.0321e6
"""GARANTIAS BANCOMEXT""","""BANCOMEXT BBVA BANCOMER""","""60423958""",3999,390082,3.3546e8
"""GARANTIAS BANCOMEXT""","""BANCOMEXT-BANORTE""","""59639999""",3999,390213,1.8667e6
"""GARANTIAS BANCOMEXT""","""BANCOMEXT BBVA BANCOMER""","""60423958""",3999,390084,8.3609e7
"""GARANTIAS BANCOMEXT""","""BANCOMEXT/BANCO SABADELL, S.A.…","""162042417""",3999,390167,1.9622e7
…,…,…,…,…,…
"""GARANTIAS BANCOMEXT""","""BANCOMEXT BBVA BANCOMER""","""60423958""",3999,39897,1.9928e8
"""GARANTIAS BANCOMEXT""","""BANCOMEXT/BAJIO""","""64991057""",3999,390154,1.0413e9
"""GARANTIAS BANCOMEXT""","""BANCOMEXT/BAJIO""","""64991057""",3999,390056,6.8062e7


In [21]:
# Agrup saldo total automatica
(
    saldos_totales_vf
    .filter(
        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
        (pl.col("Producto") != "GARANTIAS BANSEFI") &
        (pl.col("Razón Social (Intermediario)") != "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
    ).group_by([
        "Producto", 
        "Razón Social (Intermediario)", 
        "INTERMEDIARIO_ID", 
        "PROGRAMA_ORIGINAL", 
        "PROGRAMA_ID"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,Razón Social (Intermediario),INTERMEDIARIO_ID,PROGRAMA_ORIGINAL,PROGRAMA_ID,SALDO_MN
str,str,str,i64,i64,f64
"""GARANTIA EMPRESARIAL""","""HSBC MEXICO""","""10000171""",3999,35794,375455.26656
"""GARANTIA SECTORIAL""","""BANAMEX""","""10000233""",3999,33735,186992.472
"""GARANTIA EMPRESARIAL""","""BANCO DEL BAJIO""","""10040030""",3999,310034,2.3485e7
"""GARANTIA SECTORIAL""","""BANCA AFIRME""","""10040062""",3999,320938,3.7250e6
"""GARANTIA EMPRESARIAL""","""BBV BANCOMER""","""10040012""",3999,310176,3.2522e9
…,…,…,…,…,…
"""GARANTIA SECTORIAL""","""BANCO SANTANDER""","""10000144""",3999,36693,1.1669e6
"""GARANTIA SECTORIAL""","""BANCO DEL BAJIO""","""10040030""",3999,320943,541666.98
"""GARANTIA SECTORIAL""","""HSBC MEXICO""","""10000171""",3999,320630,4.8606e6


In [18]:
# AgrupTaxo saldo total
agrup_taxo_saldo_total = (
    saldos_totales_vf
    .group_by([
        "Producto"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)
agrup_taxo_saldo_total.write_csv(fl_agrup_taxo_saldo_total, datetime_format="%Y-%m-%d", float_precision=6)

In [ ]:
# AgrupTaxo saldo total resto
(
    saldos_totales_vf
        .filter(
            (pl.col("Producto") == "GARANTIAS BANCOMEXT") |
            (pl.col("Producto") == "GARANTIAS SHF-LI FINANCIERO") |
            (pl.col("Producto") == "GARANTIAS BANSEFI") |
            (pl.col("Razón Social (Intermediario)") == "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
        ).group_by([
            "Producto"
            ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

In [24]:
(
    saldos_totales_vf
    .filter(
        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
        (pl.col("Producto") != "GARANTIAS BANSEFI") &
        (pl.col("Razón Social (Intermediario)") != "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
        ).group_by([
            "Producto"
            ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,SALDO_MN
str,f64
"""GARANTIA ESPECIAL""",6.2906e7
"""GARANTIA EMERGENTES""",4.7975e8
"""GARANTIA SECTORIAL""",1.8266e10
"""EMPRESA MEDIANA""",5.1653e8
"""GARANTIA FIANZA""",6.3914e9
"""GARANTIAS FEFA""",4.0824e8
"""GARANTIA EMPRESARIAL""",8.6838e10
"""GARANTIA SUBASTA""",618635.560894
